In [3]:
# transformers nltk datasets peft torch
! pip install evaluate rouge_score

In [1]:
import numpy as np 
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from datasets import Dataset,load_from_disk,load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model, TaskType, PeftConfig
import torch
#from evaluate import load

2025-07-25 17:23:43.898771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753464224.215705      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753464224.317874      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ModuleNotFoundError: No module named 'evaluate'

In [5]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["WANDB_API_KEY"] =user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "T5B-PEFT-FNHR-Entitle"

In [6]:
!wandb login

wandb: Currently logged in as: ankonbh (ankonbh-university-of-leeds) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_ACC_TOK")

In [ ]:
model_name="google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
metric = load("rouge")

In [9]:
tokenized_dataset=load_dataset("Ankonbh/Financial-News-Headlines-Reuters")

README.md:   0%|          | 0.00/639 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/6.64M [00:00<?, ?B/s]

data/val-00000-of-00001.parquet:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19661 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/4916 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8193 [00:00<?, ? examples/s]

In [3]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05, 
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

print(lora_config)

LoraConfig(task_type=<TaskType.SEQ_2_SEQ_LM: 'SEQ_2_SEQ_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=16, target_modules=None, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)


In [4]:
model = get_peft_model(model, lora_config)

In [5]:
model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096


In [20]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [22]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    f"{model_name}-PEFT-FNHR",
    eval_strategy="epoch",
    learning_rate=3e-4, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
    save_strategy="epoch",
    metric_for_best_model="eval_rougeL",
    report_to="wandb",
    save_total_limit=3,
    push_to_hub=False,
)

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [25]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: ankonbh (ankonbh-university-of-leeds) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.321000,1.967862,45.558800,21.390000,41.473300,41.498700,15.114300
2,2.232700,1.931288,45.982400,21.871700,41.876500,41.898400,15.813900
3,2.184300,1.915460,46.185900,21.953300,42.045900,42.064400,15.993100
4,2.161100,1.908943,46.394600,22.085000,42.247500,42.258100,15.938800
5,2.120500,1.902210,46.352100,22.147300,42.183400,42.208600,15.981900
6,2.103000,1.898485,46.457400,22.201800,42.310200,42.321600,16.054300


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=3690, training_loss=2.1745280609544375, metrics={'train_runtime': 3038.7118, 'train_samples_per_second': 38.821, 'train_steps_per_second': 1.214, 'total_flos': 1.2009308592208896e+16, 'train_loss': 2.1745280609544375, 'epoch': 6.0})

In [27]:
trainer.model.push_to_hub(repo_id="Ankonbh/flan-t5-base-PEFT-FNHR")

Uploading...:   0%|          | 0.00/7.10M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ankonbh/flan-t5-base-PEFT-FNHR/commit/5915ed92b9b660297b60efc7b01a77e1e79a605e', commit_message='Upload model', commit_description='', oid='5915ed92b9b660297b60efc7b01a77e1e79a605e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ankonbh/flan-t5-base-PEFT-FNHR', endpoint='https://huggingface.co', repo_type='model', repo_id='Ankonbh/flan-t5-base-PEFT-FNHR'), pr_revision=None, pr_num=None)

In [28]:
prediction_output = trainer.predict(tokenized_dataset['test'])

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [29]:
prediction_output.metrics

{'test_loss': 1.9054192304611206,
 'test_rouge1': 46.1543,
 'test_rouge2': 21.7384,
 'test_rougeL': 42.0157,
 'test_rougeLsum': 42.0307,
 'test_gen_len': 15.9956,
 'test_runtime': 254.6074,
 'test_samples_per_second': 32.179,
 'test_steps_per_second': 1.009}